Yay! Let's get started!

### Design decisions?
- Punctuation looks like it's important, so I think we should keep it? like `!!!!` could actually be indicative of something, as might `???`. Periods, maybe not so much, so I think I'm gonna remove those, commas too
- It might be better to split up contractions, so I thought that could be n i c e
- iffy on stemming and lemmatization. Vader doesn't like it, but we can try it?
    - lemmatization: spacy
- or, try two versions, one with and the other without, see which one performs better
- right, so googles pretrained word2vec exists? to use or not to use, and how?


In [1]:
# All our imports
import json
import nltk
import spacy
import re
from nltk import word_tokenize
from nltk import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [88]:
# constants should we want that

# taken from wikipedia + stackexchange answer: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
    "ain't": "am not / are not / is not / has not / have not",
    "aren't": "are not / am not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had / he would",
    "he'd've": "he would have",
    "he'll": "he shall / he will",
    "he'll've": "he shall have / he will have",
    "he's": "he has / he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how has / how is / how does",
    "I'd": "I had / I would",
    "I'd've": "I would have",
    "I'll": "I shall / I will",
    "I'll've": "I shall have / I will have",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it had / it would",
    "it'd've": "it would have",
    "it'll": "it shall / it will",
    "it'll've": "it shall have / it will have",
    "it's": "it has / it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she had / she would",
    "she'd've": "she would have",
    "she'll": "she shall / she will",
    "she'll've": "she shall have / she will have",
    "she's": "she has / she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as / so is",
    "that'd": "that would / that had",
    "that'd've": "that would have",
    "that's": "that has / that is",
    "there'd": "there had / there would",
    "there'd've": "there would have",
    "there's": "there has / there is",
    "they'd": "they had / they would",
    "they'd've": "they would have",
    "they'll": "they shall / they will",
    "they'll've": "they shall have / they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had / we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what shall / what will",
    "what'll've": "what shall have / what will have",
    "what're": "what are",
    "what's": "what has / what is",
    "what've": "what have",
    "when's": "when has / when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where has / where is",
    "where've": "where have",
    "who'll": "who shall / who will",
    "who'll've": "who shall have / who will have",
    "who's": "who has / who is",
    "who've": "who have",
    "why's": "why has / why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had / you would",
    "you'd've": "you would have",
    "you'll": "you shall / you will",
    "you'll've": "you shall have / you will have",
    "you're": "you are",
    "you've": "you have"
}

In [89]:
# a bit of set up
lemmatization_model = spacy.load('en_core_web_sm')

# Data Prep

### Tokenizing and Normalizing

In [94]:
# making a class with review_id, review_body, stars, product id and category
class parsedEntry:
    def __init__(self, review_id, sent_text, all_text, stars, product_id, product_category):
        self.review_id = review_id
        self.sent_text = sent_text
        self.all_text = all_text
        self.stars = stars
        self.product_id = product_id
        self.product_category = product_category

    def printAll(self):
        print(self.review_id)
        print("\t TEXT: " + str(self.all_text))
        #print("\t SENT TEXT: " + str(self.sent_text))
        print("\t STARS: " + str(self.stars))
        print()

In [95]:
# making a list of parsedEntries
def makeListEntries(fileName):
    
    listEntries = []

    for line in open(fileName, 'r'):
        entry = json.loads(line)

        # GETTING THE PERTINENT BITS #
        review_id = entry['review_id']
        text = entry['review_body']
        stars = entry['stars']
        product_id = entry['product_id']
        product_category = entry['product_category']

        # DEAL WITH THE TEXT PORTION
        text = text.lower()
        
        # breaking up some contractions: globally replace key with value pair
        nonCont = text
        for key in contractions:
            nonCont = re.sub(key, contractions[key], nonCont)

        sentences = sent_tokenize(nonCont)

        tokenized_sentences = []
        tokenized_all = []
        for sentence in sentences:
            # makes a list type dealio
            tokens = lemmatization_model(sentence)

           # list just for this sentence
            sentenceList = []

            for token in tokens:
                if (token.lemma_ != "." and token.lemma_ != ","):
                    tokenized_all.append(token.lemma_)
                    sentenceList.append(token.lemma_)

            # add sentenceList to total sentences
            tokenized_sentences.append(sentenceList)

        newEntry = parsedEntry(review_id, tokenized_sentences, tokenized_all, stars, product_id, product_category)

        listEntries.append(newEntry)
    return listEntries
    

In [96]:
trainSet = makeListEntries("dataset/smol_train.json")
testSet = makeListEntries("dataset/smol_test.json")

In [97]:
for i in range(10):
    trainSet[i].printAll()
    testSet[i].printAll()

en_0964290
	 TEXT: ['arrive', 'broken', 'manufacturer', 'defect', 'two', 'of', 'the', 'leg', 'of', 'the', 'base', 'be', 'not', 'completely', 'form', 'so', 'there', 'be', 'no', 'way', 'to', 'insert', 'the', 'caster', 'I', 'unpackage', 'the', 'entire', 'chair', 'and', 'hardware', 'before', 'notice', 'this', 'so', 'I', "'ll", 'spend', 'twice', 'the', 'amount', 'of', 'time', 'box', 'up', 'the', 'whole', 'useless', 'thing', 'and', 'send', 'it', 'back', 'with', 'a', '1', '-', 'star', 'review', 'of', 'part', 'of', 'a', 'chair', 'I', 'never', 'get', 'to', 'sit', 'in', 'I', 'will', 'go', 'so', 'far', 'as', 'to', 'include', 'a', 'picture', 'of', 'what', 'their', 'injection', 'molding', 'and', 'quality', 'assurance', 'process', 'miss', 'though', 'I', 'will', 'be', 'hesitant', 'to', 'buy', 'again', 'it', 'make', 'I', 'wonder', 'if', 'there', 'be', 'not', '/', 'be', 'not', 'miss', 'structure', 'and', 'support', 'that', 'do', 'not', 'impede', 'the', 'assembly', 'process']
	 STARS: 1

en_0199937
	 TE

### Vectorizing

In [98]:
# makes the list of words a string, adds that to a list
def makeListText(dataSet):
    resList = []
    for entry in dataSet:
        resList.append(" ".join(entry.all_text))
    return resList

# deal with target (the stars) as well
def makeListStars(dataSet):
    resList = []
    for entry in dataSet:
        resList.append(entry.stars)
    return resList

# data
listTrainText = makeListText(trainSet)
listTestText = makeListText(testSet)

# target
listTrainStars = makeListStars(trainSet)
listTestStars = makeListStars(testSet)

#### CountVectorizer

In [99]:
# could do CountVectorizer
cv = CountVectorizer(stop_words = 'english')

trainCVMatr = cv.fit_transform(listTrainText)
testCVMatr = cv.transform(listTrainText)

#print(trainCVMatr)
#print(testCVMatr)

#### TdidfVectorizer

In [100]:
# could do TfidfVectorizer
tv = TfidfVectorizer()

trainTVMatr = cv.fit_transform(listTrainText)
testTVMatr = cv.transform(listTrainText)

#print(trainTVMatr)
#print(testTVMatr)